In [2]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [4]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [5]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [6]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [7]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [8]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [9]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [10]:

token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [69]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(150))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1500, 300)         13049100  
_________________________________________________________________
simple_rnn_11 (SimpleRNN)    (None, 150)               67650     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 151       
Total params: 13,116,901
Trainable params: 13,116,901
Non-trainable params: 0
_________________________________________________________________
CPU times: user 201 ms, sys: 34.7 ms, total: 235 ms
Wall time: 234 ms


In [70]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=16*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/5
75/75 [==============================] - 3s 45ms/step - accuracy: 0.9011 - loss: 0.3069
Epoch 2/5
75/75 [==============================] - 3s 45ms/step - accuracy: 0.9161 - loss: 0.2180
Epoch 3/5
75/75 [==============================] - 3s 46ms/step - accuracy: 0.9664 - loss: 0.0979
Epoch 4/5
75/75 [==============================] - 3s 46ms/step - accuracy: 0.9919 - loss: 0.0298
Epoch 5/5
75/75 [==============================] - 3s 45ms/step - accuracy: 0.9985 - loss: 0.0088


In [71]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores[:2401],yvalid)))

Auc: 0.88%


In [72]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores[:2401],yvalid)})
scores_model[0]={'Model': 'SimpleRNN','AUC_Score': roc_auc(scores[:2401],yvalid)}

In [42]:
xtrain_seq[:1]

[[664,
  65,
  7,
  19,
  2262,
  14102,
  5,
  2262,
  20439,
  6071,
  4,
  71,
  32,
  20440,
  6620,
  39,
  6,
  664,
  65,
  11,
  8,
  20441,
  1502,
  38,
  6072]]

In [43]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [05:57, 6137.13it/s]

Found 2196017 word vectors.


In [44]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 134102.00it/s]


In [45]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(150, dropout=0.25, recurrent_dropout=0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1500, 300)         13049100  
_________________________________________________________________
lstm (LSTM)                  (None, 150)               270600    
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 151       
Total params: 13,319,851
Trainable params: 270,751
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 790 ms, sys: 1.05 s, total: 1.84 s
Wall time: 2.79 s


In [46]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=32*strategy.num_replicas_in_sync)

Epoch 1/5
38/38 [==============================] - 6s 146ms/step - accuracy: 0.9137 - loss: 0.2675
Epoch 2/5
38/38 [==============================] - 4s 109ms/step - accuracy: 0.9443 - loss: 0.1570
Epoch 3/5
38/38 [==============================] - 4s 110ms/step - accuracy: 0.9493 - loss: 0.1364
Epoch 4/5
38/38 [==============================] - 4s 109ms/step - accuracy: 0.9550 - loss: 0.1237
Epoch 5/5
38/38 [==============================] - 4s 110ms/step - accuracy: 0.9585 - loss: 0.1158


In [47]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores[:2401],yvalid)))

Auc: 0.97%


In [49]:
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores[:2401],yvalid)})

In [54]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1500, 300)         13049100  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1500, 300)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               541800    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 301       
Total params: 13,591,201
Trainable params: 542,101
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 581 ms, sys: 867 ms, total: 1.45 s
Wall time: 3.09 s


In [55]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 4s 224ms/step - accuracy: 0.8805 - loss: 0.3355
Epoch 2/5
19/19 [==============================] - 2s 107ms/step - accuracy: 0.9304 - loss: 0.2136
Epoch 3/5
19/19 [==============================] - 2s 107ms/step - accuracy: 0.9450 - loss: 0.1603
Epoch 4/5
19/19 [==============================] - 2s 107ms/step - accuracy: 0.9518 - loss: 0.1312
Epoch 5/5
19/19 [==============================] - 2s 107ms/step - accuracy: 0.9550 - loss: 0.1189


In [56]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores[:2401],yvalid)))

Auc: 0.98%


In [53]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores[:2401],yvalid)})

In [73]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.8822406575102635},
 {'Model': 'LSTM', 'AUC_Score': 0.9744598762305015},
 {'Model': 'GRU', 'AUC_Score': 0.9757486352528277}]

In [78]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(200, dropout=0.25, recurrent_dropout=0.25)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 1500, 300)         13049100  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 400)               801600    
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 401       
Total params: 13,851,101
Trainable params: 802,001
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 1.18 s, sys: 1.05 s, total: 2.23 s
Wall time: 2.19 s


In [79]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=32*strategy.num_replicas_in_sync)

Epoch 1/5
38/38 [==============================] - 11s 284ms/step - accuracy: 0.9051 - loss: 0.2821
Epoch 2/5
38/38 [==============================] - 9s 233ms/step - accuracy: 0.9422 - loss: 0.1616
Epoch 3/5
38/38 [==============================] - 9s 233ms/step - accuracy: 0.9509 - loss: 0.1399
Epoch 4/5
38/38 [==============================] - 9s 233ms/step - accuracy: 0.9538 - loss: 0.1292
Epoch 5/5
38/38 [==============================] - 9s 233ms/step - accuracy: 0.9505 - loss: 0.1333


In [80]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores[:2401],yvalid)))

Auc: 0.97%


In [81]:
scores_model.append({'Model': 'Bi-directional LSTM','AUC_Score': roc_auc(scores[:2401],yvalid)})

In [82]:
# Visualization of Results obtained from various Deep learning models
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

,Model,AUC_Score
2,GRU,0.975749
1,LSTM,0.974460
3,Bi-directional LSTM,0.966816
0,SimpleRNN,0.882241


In [83]:
fig = go.Figure(go.Funnelarea(
    text =results.Model,
    values = results.AUC_Score,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

In [84]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers

from tokenizers import BertWordPieceTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [85]:
# LOADING THE DATA

train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

Encoder FOr DATA for understanding waht encode batch does read documentation of hugging face tokenizer :
https://huggingface.co/transformers/main_classes/tokenizer.html here

In [86]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [87]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 6
BATCH_SIZE = 18 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [88]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False, wordpieces_prefix=##)

In [89]:
x_train = fast_encode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train1.toxic.values
y_valid = valid.toxic.values

100%|██████████| 250/250 [00:10<00:00, 23.36it/s]


In [90]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [48]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [49]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 33.4 s, sys: 11.3 s, total: 44.7 s
Wall time: 51.5 s


In [50]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3
1746/1746 [==============================] - 176s 101ms/step - loss: 0.1320 - accuracy: 0.9484 - val_loss: 0.4166 - val_accuracy: 0.8491
Epoch 2/3
1746/1746 [==============================] - 165s 95ms/step - loss: 0.0967 - accuracy: 0.9604 - val_loss: 0.4681 - val_accuracy: 0.8485
Epoch 3/3
1746/1746 [==============================] - 165s 95ms/step - loss: 0.0833 - accuracy: 0.9657 - val_loss: 0.6499 - val_accuracy: 0.8480


In [51]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
62/62 [==============================] - 6s 91ms/step - loss: 0.3367 - accuracy: 0.8595
Epoch 2/6
62/62 [==============================] - 6s 91ms/step - loss: 0.2438 - accuracy: 0.8943
Epoch 3/6
62/62 [==============================] - 6s 91ms/step - loss: 0.1829 - accuracy: 0.9190
Epoch 4/6
62/62 [==============================] - 6s 91ms/step - loss: 0.1261 - accuracy: 0.9449
Epoch 5/6
62/62 [==============================] - 6s 91ms/step - loss: 0.0822 - accuracy: 0.9668
Epoch 6/6
62/62 [==============================] - 6s 91ms/step - loss: 0.0764 - accuracy: 0.9694
